In [125]:
# -*- coding: utf-8 -*-
"""
Created on 05/13, 2023
@author: WillyF

"""
# https://data.world/etocco/nba-player-stats
# https://www.basketball-reference.com/leagues/NBA_1997_per_game.html
# https://www.statscrew.com/basketball/t-WSB




'\nCreated on 05/13, 2023\n@author: WillyF\n\n'

In [126]:
import pandas as pd
import urllib.request
import json
import numpy as np
# from glob import glob
import os
from warnings import filterwarnings
filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [127]:
stats_append_1996_97 = pd.read_csv('1996-97_Stats_Append.txt').fillna(0.000).drop(columns=["Player-additional"])
stats_append_1996_97["Season"] = "1996-97"
stats_append_1996_97["MVP"] = False
stats_append_1996_97.loc[stats_append_1996_97["Player"].str.contains("Karl Malone", regex=False),"MVP"] = True

stats_append_2022_23 = pd.read_csv('2022-23_Stats_Append.txt').fillna(0.000).drop(columns=["Player-additional"])
stats_append_2022_23["Season"] = "2022-23"
stats_append_2022_23["MVP"] = False
stats_append_2022_23.loc[stats_append_2022_23["Player"].str.contains("Joel Embiid", regex=False),"MVP"] = True

In [128]:
nba_team_list = pd.read_csv('https://raw.githubusercontent.com/LeBronWilly/NBA_Player_Stats/main/NBA_Team_List.csv').fillna(0.000)
nba_player_df = pd.read_csv('https://query.data.world/s/ty4piswbrsp54noyh2bdvrjecv24ta?dws=00000').fillna(0.000)
nba_player_df = nba_player_df.append([stats_append_1996_97, stats_append_2022_23], ignore_index=True)
nba_player_df["BHOF"] = False
nba_player_df.loc[nba_player_df["Player"].str.contains("*", regex=False),"BHOF"] = True
nba_player_df["Player"] = nba_player_df["Player"].str.strip("*")
nba_player_df = nba_player_df.merge(nba_team_list, how="left", on="Tm", validate="many_to_one").drop(columns=["Rk"])
nba_player_df = nba_player_df.sort_values(by=['Season', 'Player']).reset_index(drop=True)
nba_player_df.insert(3, 'Season', nba_player_df.pop('Season'))
nba_player_df.insert(4, 'Team', nba_player_df.pop('Team'))
nba_player_df.insert(5, 'Season_MVP', nba_player_df.pop('MVP'))
nba_player_df.insert(6, 'BHOF', nba_player_df.pop('BHOF'))
nba_player_df.insert(len(nba_player_df.columns)-1, 'Tm', nba_player_df.pop('Tm'))
nba_player_df

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm
0,A.C. Green,SF,32,1996-97,Phoenix Suns,False,False,82,36,25.8,2.6,5.4,0.484,0.2,0.6,0.269,2.5,4.8,0.513,0.500,2.0,2.9,0.709,2.0,4.7,6.8,0.9,0.5,0.3,1.0,1.7,7.5,PHO
1,Aaron McKie,SG,23,1996-97,Portland Trail Blazers,False,False,81,73,27.9,4.2,8.9,0.467,0.5,1.4,0.325,3.7,7.5,0.494,0.493,1.9,2.5,0.764,1.1,2.7,3.8,2.5,1.1,0.3,1.7,2.5,10.7,POR
2,Acie Earl,C,25,1996-97,Toronto Raptors,False,False,42,7,15.6,2.8,6.6,0.424,0.0,0.1,0.000,2.8,6.5,0.429,0.424,2.0,2.7,0.719,1.2,1.9,3.1,0.6,0.4,0.9,1.2,1.7,7.5,TOR
3,Adam Keefe,PF,25,1996-97,Utah Jazz,False,False,82,0,20.8,2.2,4.2,0.520,0.0,0.0,0.000,2.2,4.2,0.526,0.520,1.7,2.5,0.692,2.1,3.4,5.5,0.8,0.6,0.5,1.1,2.1,6.1,UTA
4,Adrian Caldwell,PF,29,1996-97,Indiana Pacers,False,False,51,1,6.4,0.9,1.6,0.554,0.0,0.0,0.000,0.9,1.6,0.554,0.554,0.4,0.7,0.500,0.8,1.3,2.2,0.1,0.2,0.1,0.7,1.4,2.2,IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15792,Zach Collins,C,25,2022-23,San Antonio Spurs,False,False,63,26,22.9,4.5,8.7,0.518,0.9,2.3,0.374,3.6,6.4,0.571,0.568,1.7,2.3,0.761,1.8,4.5,6.4,2.9,0.6,0.8,2.0,3.2,11.6,SAS
15793,Zach LaVine,SG,27,2022-23,Chicago Bulls,False,False,77,77,35.9,8.7,18.0,0.485,2.6,7.1,0.375,6.1,11.0,0.556,0.558,4.7,5.6,0.848,0.5,3.9,4.5,4.2,0.9,0.2,2.5,2.1,24.8,CHI
15794,Zeke Nnaji,PF,22,2022-23,Denver Nuggets,False,False,53,5,13.7,2.1,3.7,0.561,0.3,1.2,0.262,1.8,2.5,0.710,0.605,0.8,1.2,0.645,1.2,1.4,2.6,0.3,0.3,0.4,0.6,2.0,5.2,DEN
15795,Ziaire Williams,SF,21,2022-23,Memphis Grizzlies,False,False,37,4,15.2,2.3,5.3,0.429,0.7,2.6,0.258,1.6,2.7,0.596,0.492,0.5,0.6,0.773,0.4,1.7,2.1,0.9,0.4,0.2,1.0,1.6,5.7,MEM


In [129]:
nba_player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15797 entries, 0 to 15796
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      15797 non-null  object 
 1   Pos         15797 non-null  object 
 2   Age         15797 non-null  int64  
 3   Season      15797 non-null  object 
 4   Team        15797 non-null  object 
 5   Season_MVP  15797 non-null  bool   
 6   BHOF        15797 non-null  bool   
 7   G           15797 non-null  int64  
 8   GS          15797 non-null  int64  
 9   MP          15797 non-null  float64
 10  FG          15797 non-null  float64
 11  FGA         15797 non-null  float64
 12  FG%         15797 non-null  float64
 13  3P          15797 non-null  float64
 14  3PA         15797 non-null  float64
 15  3P%         15797 non-null  float64
 16  2P          15797 non-null  float64
 17  2PA         15797 non-null  float64
 18  2P%         15797 non-null  float64
 19  eFG%        15797 non-nul

In [130]:
# nba_player_df.to_excel("nba_player_df.xlsx")
# nba_player_df.to_csv("nba_player_df.csv")

In [131]:
nba_player_df[nba_player_df["Player"].str.contains("ulionis", regex=False)]

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm
543,Šarūnas Marčiulionis,SG,31,1996-97,Sacramento Kings,False,True,53,0,19.6,3.3,7.3,0.452,1.2,3.0,0.408,2.1,4.4,0.483,0.535,2.9,3.8,0.775,0.4,1.1,1.5,2.2,1.0,0.1,1.8,2.1,10.8,SAC


In [132]:
nba_player_df[nba_player_df["Player"].str.contains("?", regex=False)]

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm


In [133]:
nba_player_df[nba_player_df["Player"].str.contains("vis Bert", regex=False)]

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm
11208,Dāvis Bertāns,PF,24,2016-17,San Antonio Spurs,False,False,67,6,12.1,1.5,3.5,0.440,1.0,2.6,0.399,0.5,0.9,0.557,0.588,0.4,0.5,0.824,0.3,1.1,1.5,0.7,0.3,0.4,0.5,1.1,4.5,SAS
11801,Dāvis Bertāns,PF,25,2017-18,San Antonio Spurs,False,False,77,10,14.1,2.1,4.8,0.440,1.2,3.3,0.373,0.9,1.5,0.588,0.568,0.5,0.6,0.816,0.2,1.8,2.0,1.0,0.3,0.4,0.5,1.2,5.9,SAS
12458,Dāvis Bertāns,PF,26,2018-19,San Antonio Spurs,False,False,76,12,21.5,2.7,6.0,0.450,1.9,4.4,0.429,0.8,1.5,0.513,0.610,0.7,0.8,0.883,0.3,3.2,3.5,1.3,0.5,0.4,0.6,1.8,8.0,SAS
13160,Dāvis Bertāns,PF,27,2019-20,Washington Wizards,False,False,54,4,29.3,4.9,11.3,0.434,3.7,8.7,0.424,1.2,2.6,0.471,0.598,1.9,2.3,0.852,0.6,3.9,4.5,1.7,0.7,0.6,1.1,2.6,15.4,WAS
13817,Dāvis Bertāns,PF,28,2020-21,Washington Wizards,False,False,57,7,25.7,3.4,8.4,0.404,3.0,7.5,0.395,0.4,0.9,0.480,0.581,1.8,2.0,0.870,0.3,2.6,2.9,0.9,0.6,0.2,0.6,2.4,11.5,WAS
14541,Dāvis Bertāns,PF,29,2021-22,Two Other Teams,False,False,56,0,14.4,1.8,4.9,0.360,1.4,4.1,0.335,0.4,0.8,0.489,0.500,0.6,0.7,0.900,0.2,1.9,2.1,0.6,0.3,0.2,0.3,1.5,5.6,TOT
14542,Dāvis Bertāns,PF,29,2021-22,Washington Wizards,False,False,34,0,14.7,1.8,5.0,0.351,1.4,4.2,0.319,0.4,0.8,0.519,0.485,0.8,0.9,0.933,0.2,1.6,1.8,0.5,0.3,0.2,0.4,1.6,5.7,WAS
14543,Dāvis Bertāns,PF,29,2021-22,Dallas Mavericks,False,False,22,0,13.9,1.8,4.7,0.375,1.4,3.9,0.360,0.4,0.8,0.444,0.524,0.4,0.5,0.800,0.3,2.3,2.5,0.7,0.3,0.3,0.2,1.4,5.3,DAL
15292,Dāvis Bertāns,PF,30,2022-23,Dallas Mavericks,False,False,45,1,10.9,1.5,3.6,0.431,1.2,3.1,0.390,0.3,0.4,0.737,0.603,0.3,0.3,0.867,0.2,1.0,1.2,0.5,0.2,0.2,0.2,1.2,4.6,DAL


In [134]:
nba_player_df[nba_player_df["Player"].str.contains("Alperen", regex=False)] 

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm
14337,Alperen Şengün,C,19,2021-22,Houston Rockets,False,False,72,13,20.7,3.5,7.3,0.474,0.4,1.6,0.248,3.1,5.7,0.539,0.502,2.3,3.2,0.711,1.9,3.5,5.5,2.6,0.8,0.9,2.0,3.0,9.6,HOU
15135,Alperen Şengün,C,20,2022-23,Houston Rockets,False,False,75,72,28.9,5.9,10.7,0.553,0.3,0.8,0.333,5.6,9.9,0.570,0.565,2.7,3.8,0.715,3.2,5.8,9.0,3.9,0.9,0.9,2.6,3.4,14.8,HOU


In [135]:
nba_player_df[nba_player_df["Player"].str.contains("Boban Marjanov", regex=False)]

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm
10518,Boban Marjanović,C,27,2015-16,San Antonio Spurs,False,False,54,4,9.4,1.9,3.2,0.603,0.0,0.0,0.000,1.9,3.2,0.603,0.603,1.6,2.1,0.763,1.4,2.2,3.6,0.4,0.2,0.4,0.5,1.0,5.5,SAS
11091,Boban Marjanović,C,28,2016-17,Detroit Pistons,False,False,35,0,8.4,2.1,3.8,0.545,0.0,0.0,0.000,2.1,3.8,0.545,0.545,1.3,1.7,0.810,1.3,2.4,3.7,0.3,0.2,0.3,0.3,0.7,5.5,DET
11671,Boban Marjanović,C,29,2017-18,Two Other Teams,False,False,39,1,8.6,2.0,3.8,0.534,0.0,0.0,0.000,2.0,3.8,0.534,0.534,2.0,2.5,0.794,1.2,2.5,3.7,0.6,0.3,0.3,0.9,1.3,6.0,TOT
11672,Boban Marjanović,C,29,2017-18,Detroit Pistons,False,False,19,1,9.0,2.2,4.2,0.519,0.0,0.0,0.000,2.2,4.2,0.519,0.519,1.9,2.4,0.800,0.8,2.2,3.0,0.7,0.2,0.3,1.2,1.5,6.2,DET
11673,Boban Marjanović,C,29,2017-18,Los Angeles Clippers,False,False,20,0,8.3,1.9,3.5,0.551,0.0,0.0,0.000,1.9,3.5,0.551,0.551,2.1,2.6,0.788,1.7,2.7,4.4,0.4,0.3,0.3,0.7,1.2,5.9,LAC
12327,Boban Marjanović,C,30,2018-19,Two Other Teams,False,False,58,12,11.7,2.8,4.5,0.615,0.1,0.2,0.400,2.7,4.3,0.624,0.623,1.7,2.3,0.748,1.5,3.1,4.6,0.9,0.3,0.5,1.0,1.6,7.3,TOT
12328,Boban Marjanović,C,30,2018-19,Los Angeles Clippers,False,False,36,9,10.4,2.4,3.9,0.607,0.0,0.1,0.000,2.4,3.8,0.616,0.607,2.0,2.6,0.758,1.5,2.7,4.2,0.6,0.3,0.5,1.0,1.6,6.7,LAC
12329,Boban Marjanović,C,30,2018-19,Philadelphia 76ers,False,False,22,3,13.9,3.4,5.5,0.625,0.2,0.4,0.500,3.2,5.1,0.634,0.642,1.2,1.6,0.722,1.5,3.7,5.1,1.5,0.2,0.5,1.0,1.5,8.2,PHI
13026,Boban Marjanović,C,31,2019-20,Dallas Mavericks,False,False,44,5,9.6,2.8,4.8,0.573,0.1,0.4,0.235,2.7,4.4,0.603,0.583,1.0,1.3,0.754,1.5,3.0,4.5,0.5,0.2,0.2,0.7,1.3,6.6,DAL
13674,Boban Marjanović,C,32,2020-21,Dallas Mavericks,False,False,33,3,8.2,1.9,3.7,0.508,0.0,0.2,0.125,1.8,3.5,0.535,0.512,0.9,1.2,0.816,1.4,2.5,3.9,0.3,0.1,0.2,0.4,0.9,4.7,DAL


In [136]:
nba_player_df[nba_player_df["Player"]=="Kobe Bryant"]

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm
877,Kobe Bryant,SF,19,1997-98,Los Angeles Lakers,False,True,79,1,26.0,4.9,11.6,0.428,0.9,2.8,0.341,4.0,8.8,0.456,0.469,4.6,5.8,0.794,1.0,2.1,3.1,2.5,0.9,0.5,2.0,2.3,15.4,LAL
1374,Kobe Bryant,SG,20,1998-99,Los Angeles Lakers,False,True,50,50,37.9,7.2,15.6,0.465,0.5,2.0,0.267,6.7,13.6,0.494,0.482,4.9,5.8,0.839,1.1,4.2,5.3,3.8,1.4,1.0,3.1,3.1,19.9,LAL
1859,Kobe Bryant,SG,21,1999-00,Los Angeles Lakers,False,True,66,62,38.2,8.4,17.9,0.468,0.7,2.2,0.319,7.7,15.7,0.489,0.488,5.0,6.1,0.821,1.6,4.7,6.3,4.9,1.6,0.9,2.8,3.3,22.5,LAL
2365,Kobe Bryant,SG,22,2000-01,Los Angeles Lakers,False,True,68,68,40.9,10.3,22.2,0.464,0.9,2.9,0.305,9.4,19.3,0.489,0.484,7.0,8.2,0.853,1.5,4.3,5.9,5.0,1.7,0.6,3.2,3.3,28.5,LAL
2874,Kobe Bryant,SG,23,2001-02,Los Angeles Lakers,False,True,80,80,38.3,9.4,20.0,0.469,0.4,1.7,0.250,9.0,18.3,0.489,0.479,6.1,7.4,0.829,1.4,4.1,5.5,5.5,1.5,0.4,2.8,2.9,25.2,LAL
3355,Kobe Bryant,SG,24,2002-03,Los Angeles Lakers,False,True,82,82,41.5,10.6,23.5,0.451,1.5,4.0,0.383,9.1,19.5,0.465,0.483,7.3,8.7,0.843,1.3,5.6,6.9,5.9,2.2,0.8,3.5,2.7,30.0,LAL
3871,Kobe Bryant,SG,25,2003-04,Los Angeles Lakers,False,True,65,64,37.6,7.9,18.1,0.438,1.1,3.3,0.327,6.8,14.8,0.463,0.468,7.0,8.2,0.852,1.6,3.9,5.5,5.1,1.7,0.4,2.6,2.7,24.0,LAL
4457,Kobe Bryant,SG,26,2004-05,Los Angeles Lakers,False,True,66,66,40.7,8.7,20.1,0.433,2.0,5.9,0.339,6.7,14.2,0.472,0.482,8.2,10.1,0.816,1.4,4.5,5.9,6.0,1.3,0.8,4.1,2.6,27.6,LAL
5009,Kobe Bryant,SG,27,2005-06,Los Angeles Lakers,False,True,80,80,41.0,12.2,27.2,0.450,2.3,6.5,0.347,10.0,20.7,0.482,0.491,8.7,10.2,0.850,0.9,4.4,5.3,4.5,1.8,0.4,3.1,2.9,35.4,LAL
5550,Kobe Bryant,SG,28,2006-07,Los Angeles Lakers,False,True,77,77,40.8,10.6,22.8,0.463,1.8,5.2,0.344,8.8,17.6,0.497,0.502,8.7,10.0,0.868,1.0,4.7,5.7,5.4,1.4,0.5,3.3,2.7,31.6,LAL


In [137]:
nba_player_df[nba_player_df["Player"]=="LeBron James"]

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm
3879,LeBron James,SG,19,2003-04,Cleveland Cavaliers,False,False,79,79,39.5,7.9,18.9,0.417,0.8,2.7,0.290,7.1,16.1,0.438,0.438,4.4,5.8,0.754,1.3,4.2,5.5,5.9,1.6,0.7,3.5,1.9,20.9,CLE
4468,LeBron James,SF,20,2004-05,Cleveland Cavaliers,False,False,80,80,42.4,9.9,21.1,0.472,1.4,3.9,0.351,8.6,17.2,0.499,0.504,6.0,8.0,0.750,1.4,6.0,7.4,7.2,2.2,0.7,3.3,1.8,27.2,CLE
5019,LeBron James,SF,21,2005-06,Cleveland Cavaliers,False,False,79,79,42.5,11.1,23.1,0.480,1.6,4.8,0.335,9.5,18.3,0.518,0.515,7.6,10.3,0.738,0.9,6.1,7.0,6.6,1.6,0.8,3.3,2.3,31.4,CLE
5560,LeBron James,SF,22,2006-07,Cleveland Cavaliers,False,False,78,78,40.9,9.9,20.8,0.476,1.3,4.0,0.319,8.6,16.8,0.513,0.507,6.3,9.0,0.698,1.1,5.7,6.7,6.0,1.6,0.7,3.2,2.2,27.3,CLE
6107,LeBron James,SF,23,2007-08,Cleveland Cavaliers,False,False,75,74,40.4,10.6,21.9,0.484,1.5,4.8,0.315,9.1,17.1,0.531,0.518,7.3,10.3,0.712,1.8,6.1,7.9,7.2,1.8,1.1,3.4,2.2,30.0,CLE
6686,LeBron James,SF,24,2008-09,Cleveland Cavaliers,True,False,81,81,37.7,9.7,19.9,0.489,1.6,4.7,0.344,8.1,15.2,0.535,0.530,7.3,9.4,0.780,1.3,6.3,7.6,7.2,1.7,1.1,3.0,1.7,28.4,CLE
7268,LeBron James,SF,25,2009-10,Cleveland Cavaliers,True,False,76,76,39.0,10.1,20.1,0.503,1.7,5.1,0.333,8.4,15.0,0.560,0.545,7.8,10.2,0.767,0.9,6.4,7.3,8.6,1.6,1.0,3.4,1.6,29.7,CLE
7875,LeBron James,SF,26,2010-11,Miami Heat,False,False,79,79,38.8,9.6,18.8,0.510,1.2,3.5,0.330,8.4,15.3,0.552,0.541,6.4,8.4,0.759,1.0,6.5,7.5,7.0,1.6,0.6,3.6,2.1,26.7,MIA
8458,LeBron James,SF,27,2011-12,Miami Heat,True,False,62,62,37.5,10.0,18.9,0.531,0.9,2.4,0.362,9.1,16.5,0.556,0.554,6.2,8.1,0.771,1.5,6.4,7.9,6.2,1.9,0.8,3.4,1.5,27.1,MIA
9018,LeBron James,PF,28,2012-13,Miami Heat,True,False,76,76,37.9,10.1,17.8,0.565,1.4,3.3,0.406,8.7,14.5,0.602,0.603,5.3,7.0,0.753,1.3,6.8,8.0,7.3,1.7,0.9,3.0,1.4,26.8,MIA


In [138]:
nba_player_df[(nba_player_df["Team"]=="Los Angeles Lakers") & (nba_player_df["Season"]=="2022-23")]

,Player,Pos,Age,Season,Team,Season_MVP,BHOF,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm
15142,Anthony Davis,C,29,2022-23,Los Angeles Lakers,False,False,56,54,34.0,9.7,17.2,0.563,0.3,1.3,0.257,9.3,15.9,0.589,0.573,6.2,7.9,0.784,3.5,9.1,12.5,2.6,1.1,2.0,2.2,2.6,25.9,LAL
15146,Austin Reaves,SG,24,2022-23,Los Angeles Lakers,False,False,64,22,28.8,4.0,7.7,0.529,1.3,3.4,0.398,2.7,4.3,0.631,0.616,3.6,4.1,0.864,0.5,2.5,3.0,3.4,0.5,0.3,1.5,1.7,13.0,LAL
15209,Cole Swider,SF,23,2022-23,Los Angeles Lakers,False,False,7,0,5.9,0.4,1.3,0.333,0.4,1.1,0.375,0.0,0.1,0.000,0.500,0.0,0.0,0.000,0.1,0.9,1.0,0.6,0.0,0.0,0.0,0.4,1.3,LAL
15215,D'Angelo Russell,PG,26,2022-23,Los Angeles Lakers,False,False,17,17,30.9,6.3,13.0,0.484,2.7,6.5,0.414,3.6,6.5,0.555,0.588,2.1,2.9,0.735,0.4,2.5,2.9,6.1,0.6,0.5,2.3,1.4,17.4,LAL
15220,Damian Jones,C,27,2022-23,Los Angeles Lakers,False,False,22,1,8.0,0.9,1.7,0.541,0.0,0.1,0.000,0.9,1.5,0.588,0.541,0.7,0.9,0.750,0.8,1.7,2.5,0.2,0.1,0.5,0.4,0.8,2.5,LAL
15243,Davon Reed,SG,27,2022-23,Los Angeles Lakers,False,False,8,0,3.4,0.4,0.5,0.750,0.1,0.3,0.500,0.3,0.3,1.000,0.875,0.1,0.5,0.250,0.1,0.4,0.5,0.5,0.3,0.0,0.0,0.4,1.0,LAL
15255,Dennis Schröder,PG,29,2022-23,Los Angeles Lakers,False,False,66,50,30.1,4.1,9.8,0.415,1.1,3.4,0.329,3.0,6.4,0.461,0.472,3.3,3.8,0.857,0.3,2.2,2.5,4.5,0.8,0.2,1.7,2.2,12.6,LAL
15400,Jarred Vanderbilt,PF,23,2022-23,Los Angeles Lakers,False,False,26,24,24.0,2.8,5.4,0.529,0.4,1.3,0.303,2.5,4.1,0.598,0.564,1.1,1.4,0.784,1.9,4.8,6.7,1.6,1.2,0.2,1.1,2.2,7.2,LAL
15460,Juan Toscano-Anderson,SF,29,2022-23,Los Angeles Lakers,False,False,30,7,12.2,1.1,2.2,0.500,0.1,0.7,0.200,1.0,1.5,0.630,0.530,0.4,0.5,0.733,0.5,1.6,2.0,0.8,0.3,0.2,0.6,1.5,2.7,LAL
15489,Kendrick Nunn,SG,27,2022-23,Los Angeles Lakers,False,False,39,2,13.5,2.6,6.4,0.406,1.1,3.2,0.325,1.6,3.2,0.488,0.488,0.4,0.5,0.810,0.1,1.3,1.4,0.9,0.3,0.1,0.8,0.9,6.7,LAL


In [139]:
sorted(set(nba_player_df["Team"]))

['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Orleans Pelicans',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Toronto Raptors',
 'Two Other Teams',
 'Utah Jazz',
 'Washington Wizards']

In [140]:
# sorted(set(nba_player_df["Player"]))

In [141]:
# sorted(set(nba_player_df["Tm"]))

In [142]:
sorted(set(nba_player_df["Season"]))

['1996-97',
 '1997-98',
 '1998-99',
 '1999-00',
 '2000-01',
 '2001-02',
 '2002-03',
 '2003-04',
 '2004-05',
 '2005-06',
 '2006-07',
 '2007-08',
 '2008-09',
 '2009-10',
 '2010-11',
 '2011-12',
 '2012-13',
 '2013-14',
 '2014-15',
 '2015-16',
 '2016-17',
 '2017-18',
 '2018-19',
 '2019-20',
 '2020-21',
 '2021-22',
 '2022-23']